# Open Loop Operation

An example of scanning over LED brightnesses and plotting the result in the photodiode

In [1]:
import numpy as np
import epics
from epics import PV
from time import sleep

## Setup

### Set all six LEDs to flash in unison

Create a six-element list of LED-duration process variables

In [2]:
LEDdurs = [None]*6
for i in range(6):
    LEDdurs[i] = PV('LEDS:CH' + str(i) + ':dur.VAL')

Set all LEDs to flash for the same duration: five hundred milliseconds.

In [3]:
for LEDdur in LEDdurs:
    LEDdur.put(500e3)

### Initialize all six LEDs at their maximum brightness

Create a six-element list of LED-brightness process variables

In [4]:
LEDbrigs = [None]*6
for i in range(6):
    LEDbrigs[i] = PV('LEDS:CH' + str(i) + ':brig.VAL')

Set all LEDs to the same brightness level: 255 (out of 255)

In [5]:
for LEDbrig in LEDbrigs:
    LEDbrig.put(255)

### Ensure the shutter is open throughout LED flash

Specify that the shutter should wait **a full 1.5 seconds** to start closing after opening

In [6]:
epics.caput("SHUTTER:dur", 1.5e6)

1

Reduce the overall repetition rate of the system to once every three seconds

In [7]:
epics.caput("PULSEGEN:reprate", 1/3.0)

1

Increase the delay time on the LED trigger to be 400 milliseconds (it started at 2 milliseconds)

In [8]:
epics.caput("PULSEGEN:CH0:delay", 400.0e3) # Set LED trigger delay to 400 ms

1

## Set up the photodetector trigger delay and exposure time

We want the photodetector to start exposure before the LEDs begin flashing, and end exposure afterwards.

We will set the delay time on the trigger to the photodetector to be 300 ms, which is 100 ms prior to the LEDs flashing.

In [9]:
epics.caput("PULSEGEN:CH1:delay", 300.0e3) # Set Photodetector trigger delay to 300 ms

1

Now, we will set the exposure length on the photodiode to a full second, to make sure we capture the entire LED flash duration.

In [10]:
epics.caput("PHOTO:dur", 1000.0e3) # Set photodetector exposure length to one second

1

## Read the photodetector value

In [11]:
epics.caget("PHOTO:data")

'DATA: 0.06364211, TRIGCNT: 727'

### Loop over brightness values

Loop over brightness demo

In [18]:
for brig in np.linspace(0, 255, 5).astype('int'):
    print("Brightness: ", brig)
    for LEDbrig in LEDbrigs:
        LEDbrig.put(brig)
    sleep(3)
    print(epics.caget("PHOTO:data"))

Brightness:  0
DATA: 0.00004400, TRIGCNT: 790
Brightness:  63
DATA: 0.00070141, TRIGCNT: 791
Brightness:  127
DATA: 0.02326047, TRIGCNT: 792
Brightness:  191
DATA: 0.02314141, TRIGCNT: 793
Brightness:  255
DATA: 0.07129035, TRIGCNT: 794
